# Exercises day 03

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import scipy
from typing import Callable
from helperfunction import *

def compute_confidence_interval_sample(data : list[float], confidence : float = 0.95) -> list[float]:
    """
        Compute the confidence interval of the mean of the data.
        :param data: list of data
        :param confidence: confidence level
        
        :return: confidence interval of the mean of the data
    """
    N = len(data)
    alpha = 1 - confidence

    data = np.array(data)
    #conf_mean = [data.mean - confidence * data.std()/np.sqrt(N), data.mean + confidence * data.std()/np.sqrt(N)]
    conf_mean = [data.mean() + data.std()/np.sqrt(N) * scipy.stats.t.cdf(alpha/2, N - 1), data.mean() + data.std()/np.sqrt(N) * scipy.stats.t.cdf(1-alpha/2, N - 1)]
    return conf_mean
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Ex01
The event for the blocking system is has the possible following event:

* Arrival of a customer 
* Customer is served
* Customer is blocked, as all $m$ service unints is occupied.

For part 1 the arrival process is modelled as a Poisson process, and the service time distribution as exponential. For a poisson process wit the mean $\lambda$ we know that the intervals between the events are exponential distributied with the same parameter. We can use this information for sampling the arrival times for the system.

In [5]:
# Define blocking system
m = 10              # service units
SERVICE_MEAN = 8    # 1/mean service time
ARRIVAL_MEAN = 1    # mean arrival time      

The implementation of the simulations follows the event-by-event principle. As the method is quite simple one could sample all the arrivals beforehand, but this approuch is not always posible for other descrete events, so we choose to follow the event-by-event principle. Here all event are stored in an event list, that contrains the eventype and is sorted by the event time.

The simulation is initiated by the first arrival, and all service units are initialized to be free. The simulations then begins in a while loop until all of the choosen arrivals has happened. Here the next event is simply the next in the event list as the list is sorted by increasing time. The simulation should imitate the steady state probability of the blocking, the simulation should run a little time before counting the blocking rate. 

An event is handled differently compared to what kind of event it is. For an arrival we first check if there is free service unit. If there is one, and service time is sampled and added to the eventlist, of not the customer er blocked and the blocked customer count is encreased by one. For an departure a service unit is freed so that the number of occupied service units is decreases by one. The probability of blocking is therefore simply the blocking count devided by the total number of arrivals. 

The code for the simulation can be found in the python file `helperfunctions`. Because you might be interested in other distributions for the arrival- and service time the simulation function takes two inputs that is the sampling from these distributions. The sampling function for the frist exercise is given as:

In [6]:
def sample_arrival_poisson_process() -> float:
    """
        Sample the arrival time from a Poisson process.
        
        :return: arrival time (as a float)
    """
    return np.random.exponential(ARRIVAL_MEAN) # TODO: check if is it not 1/arrial_mean


def sample_service_time_exponential() -> float:
    """
        Sample the service time from an exponential distribution.
        
        :return: service time (as a float)
    """
    return np.random.exponential(SERVICE_MEAN) 


The simulation gave the results:

In [8]:
block_fraction,_ = blocking_simulation()

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")


run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
The fraction of blocked customers 
[0.12090909090909091, 0.11888888888888889, 0.11747474747474747, 0.1195959595959596, 0.1195959595959596, 0.12242424242424242, 0.1111111111111111, 0.12151515151515152, 0.12131313131313132, 0.12393939393939393]
Confidence interval for the mean blocking fraction: [0.12021653884581605, 0.12054777452428872]


Exact solution

In [7]:
A = ARRIVAL_MEAN * SERVICE_MEAN

B = (A**m/np.math.factorial(m))/np.sum([(A**i)/np.math.factorial(i) for i in range(m+1)])
print(f"Exact blocking probability: {B}")

Exact blocking probability: 0.12166106425295149


The simulation is sufficiently close to the found exact solution.

## EX02
The simulation is repeated with different arrival- and service time distributions. 

### a) Erlang distributed arrival


In [9]:
def sample_arrival_erlang() -> float:
    """
        Sample the arrival time from an erlang distribution with a = 1.
        (NOTE: a = 1, is apparently the mean value in SciPys implementation, idk we asked a TA)
        
        :return: arrival time (as a float)
    """
    return scipy.stats.erlang.rvs(a = 1)

In [10]:
block_fraction = blocking_simulation(sample_arrival=sample_arrival_erlang)

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")

run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
The fraction of blocked customers 
([0.1201010101010101, 0.1395959595959596, 0.11484848484848485, 0.11616161616161616, 0.1288888888888889, 0.11, 0.1204040404040404, 0.12666666666666668, 0.12181818181818181, 0.12464646464646464], [0.9963588186372299, 0.9759524125107968, 0.9993879338430003, 1.0063089285878923, 0.9990079035552266, 1.004637119161778, 0.9962711431443216, 1.01164587253722, 0.9967830761981257, 0.9764794581208948])
Confidence interval for the mean blocking fraction: [0.7162921434517591, 0.7898553836064991]


### b) hyper exponential distribution
For sampling from the hyper exponential distribution we utilize the Composition methods from last week

In [11]:
HYPER_PROB = 0.8
HYPER_MEAN = [1/0.8333, 1/5.]

def sample_arrival_hyper_exponential() -> float:
    """
    """
    p = np.random.binomial(n=1,p= HYPER_PROB)

    if p:
        return np.random.exponential(HYPER_MEAN[0])
    else:
        return np.random.exponential(HYPER_MEAN[1])


Simulation:

In [12]:
block_fraction = blocking_simulation(sample_arrival=sample_arrival_hyper_exponential)

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")

run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
The fraction of blocked customers 
([0.13818181818181818, 0.14202020202020202, 0.12737373737373736, 0.14626262626262626, 0.13363636363636364, 0.13444444444444445, 0.1505050505050505, 0.1411111111111111, 0.13252525252525252, 0.13727272727272727], [0.9900601608885392, 0.9930300639369011, 1.0265356870308329, 0.9831166184871339, 1.0049309066250671, 0.9986562414338381, 0.9918783890534145, 1.0051623565484058, 1.001781089986869, 1.0101171831602929])
Confidence interval for the mean blocking fraction: [0.7243082173482395, 0.7968800357504441]


## Ex03

**We reuse Poisson from** *Ex01* **but then we use some new distributions for sampling the service time**
* **Constant service time**
* **Paretto distributed with $k = 1.05$ or $k = 2.05$**
* **Choose some other distributions of our own volition**

### a) Constant service time

In [12]:
sample_service_time_constant = lambda : 8

In [13]:
block_fraction = blocking_simulation(sample_service_time=sample_service_time_constant)

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")

run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
The fraction of blocked customers 
[0.1208, 0.1266, 0.1284, 0.1288, 0.1243, 0.1182, 0.1229, 0.1219, 0.1227, 0.1213]
Confidence interval for the mean blocking fraction: [0.12411488928767918, 0.1244369925590228]


### b) Pareto distributed


In [14]:
k = 1.05 

def sample_service_time_pareto() -> float:
    """
        Sample the service time from an exponential distribution.
        
        :return: service time (as a float)
    """
    U = np.random.uniform(0.0, 1.0, 1)
    beta = (k - 1) / k
    return beta * (U**(-1 / k) - 1)


def pareto_samples(k, num_samples: int):
    U = np.random.uniform(0.0, 1.0, num_samples)
    beta = 8*(k - 1) / k
    X = beta*(U**(-1/k))

    return X

In [15]:
k_list = np.linspace(1.05, 2.05, 2) # <-- might just be one of the most insane lines of code I have ever seen
k_list = np.array([1.05, 2.05])

def sample_service_time_pareto(k) -> float:
    """
        Sample the service time from an pareto distribution.
        
        :return: service time (as a float) will be above 1
    """
    return scipy.stats.pareto.rvs(k)


Simulation with the different k values

In [16]:
pareto_confs = []
for k in k_list:
    print(f"Pareto with k={k}")
    temp_lambda = lambda : sample_service_time_pareto(k)
    block_fraction = blocking_simulation(sample_service_time=temp_lambda)
    block_fraction_conf = compute_confidence_interval_sample(block_fraction)

    pareto_confs.append(block_fraction_conf)    
    
print(pareto_confs)

Pareto with k=1.05
run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
Pareto with k=2.05
run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
[[0.12144097224093167, 0.1232027714859298], [1.483543639917555e-05, 1.7802747638151312e-05]]


Does not work. 

In [17]:
k = 1.05
U = np.random.uniform(0.0, 1.0, 1000000)
beta = 8*((k - 1) / k)
samples = beta * (np.power(U, (-1 / k)) - 1)
#samples = beta * (U**(-1 / k) - 1)

print(samples.mean())

3.549655405753638
